<a href="https://colab.research.google.com/github/jugernaut/MACTI-programacionparalelo/blob/main/01_OpenMP/05_HilosDeEjecucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Hilos de ejecución</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M. en C. Miguel Angel Pérez León</i></h5>
  <h5 align="center"><i>Ayudante: Diana Laura Rojas Cristino</i></h5>
  <h5 align="center"><i>Ayudante: Rodrigo Alejandro Sánchez Morales</i></h5>
  <h5 align="center"><i>Materia: Manejo de Datos</i></h5>
</font>

# Introducción

Para esta presentación es bueno comenzar con una frase del mundo de la computación.

**“El software se ralentiza más de prisa de lo que se acelera el hardware”**

Esta frase lo que implica es que, de la mano de la creciente cantidad de datos que manejan las aplicaciones actualmente, va la necesidad de poder. Los programadores de aplicaciones, en la mayoría de los casos, no sabemos emplear la potencia del hardware y no se optimiza.

¿Qué se puede hacer al respecto?

### Multitarea

La respuesta a la pregunta anterior es, empleando el concepto de multitarea.

El concepto de multitarea radica en la capacidad de realizar varias tareas de manera simultanea, de manera tal, que una tarea que tomaba demasiado tiempo, se reduzca de manera considerable.

Pero una vez más, surge la pregunta. ¿Por qué me debe interesar que mis programas, algoritmos, etc., no demoren demasiado?.

Y en este caso la respuesta es sencilla, una aplicación o programa que toma demasiado tiempo en devolver una respuesta por lo general no se utiliza y por lo tanto no tiene sentido crearla.

# Hilo de Ejecución

El concepto de hilo (*Thread* en ingles) es una idea básica para entender como funciona la multitarea y como es que podemos agilizar el desempeño de nuestras aplicaciones.

## Definición

Hilo de Ejecución: un hilo es un trozo de código de nuestro programa que puede ser ejecutado al mismo tiempo que otro. Tambien conocido como **proceso ligero**, son un conjunto de tareas modulares que puede ser ejecutadas por el sistema operativo.

Los hilos de ejecución que comparten los mismos recursos, sumados a estos recursos, son en conjunto conocidos como un proceso. El hecho de que los hilos de ejecución de un mismo proceso compartan los recursos hace que cualquiera de estos hilos pueda modificar estos recursos.

## Anatomía del CPU

Para comprender más a detalle algunos de los estándares que usaremos a lo largo del seminario, es necesario analizar y comprender los componentes de la unida central de procesamiento (*CPU*).

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Imagenes/Introduccion/lcpu.png?raw=1" width="900">
</center>

En esta imagen, dentro del **recuadro rojo** se muestra la versión de un *CPU* (microprocesador) con un único núcleo, los elementos que componen a este *CPU* son:

*   **ALU**: unidad lógica-aritmética encargada de realizar operaciones (a nivel de bits) de tipo algebraicas $(+,-, *, /)$ y lógicas $(and, or, neg, equals)$.
*   **Registros**: mecanismo de almacenamiento de bits, de acceso muy rápido, pero muy costoso, en términos monetarios.
*   **BUS**: canal de comunicación entre el *CPU* y el resto del hardware.

Dentro del **recuadro azul**, se muestra la arquitectura de un *CPU* que puede manejar hilos (*hyperthreading*), es decir que cuenta con una única *ALU*, pero se tienen 2 registros para almacenar los respectivos datos de cada hilo:

*   **LCPU**: *CPU* lógico (virtual), se simula tener 2 *CPU's* completos.
*   **BUS**: ademas de permitir la comunicación con el resto del hardware, sirve para establecer comunicación entre hilos.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Imagenes/Introduccion/multicore.png?raw=1" width="900">
</center>

En la segunda imagen se muestra la arquitectura de un microprocesador *Quad-core*, que ademas tiene capacidades de *hyperthreading*, **4 núcleos físicos en total 8 lógicos**.

# Prestaciones de *Google Colab*

Veamos que caracteristicas para la programación en paralelo nos ofrece *google colab*.

Haciendo uso del comando *lscpu* podemos ver algunas de las características del entorno en el que se ejecutan las celdas de código de *google colab*.

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

## ¿Qué puede ejecutarse de manera simultánea?

Vamos a poner el siguiente ejemplo, imaginemos que queremos ver un listado de 100 imágenes que se descargan desde Internet, **como usuario** ¿Cuál de las dos opciones siguientes elegirías?:

* Descargar las imágenes 100 imágenes, haciendo esperar al usuario con una pantalla de “cargando” hasta que se descargan todas. Luego podrá ver el listado con las imágenes.

* Que mientras se descargan las 100 imágenes, el usuario pueda ir viendo y usando las que ya se han descargado.

* Como desarrollador la opción A es más sencilla ¿Seguro? Pero no te he preguntado cómo desarrollador, te he preguntado lo que un usuario preferiría.

## Planos

Desde el punto de vista del usuario existen dos áreas bien diferenciadas, que el desarrollador ha de tener en cuenta:

* **Primer Plano**: En este plano se ejecuta únicamente un hilo llamado “hilo principal”. Aquí hemos programado siempre, sin conocimiento de que estábamos trabajando ya con hilos. Es el hilo que trabaja con las vistas, es decir, con la interfaz gráfica que ve el usuario: botones, ventanas emergentes, campos editables, etc. También, puede ser usado para hacer cálculos u otros procesamientos complejos, aunque estos deberían de evitarse hacerse en este hilo a toda costa –salvo si es imposible que se hagan en otro hilo–.

* **Segundo Plano**: Se ejecuta todo el resto de hilos. El segundo plano tiene la característica de iniciar en el mismo momento que el primer plano. Aquí los hilos deberían de llevar las ejecuciones pesadas de la aplicación. El segundo plano el usuario no lo ve, es más, ni le interesa, para el usuario no existe.

# Diferencia entre hilo y proceso

Un proceso es aquella tarea (puede estar compuesta de subtareas) que es ejecutada por el microprocesador y en el caso de computadoras con varios procesadores, cada uno de estos CPU's puede estar ejecutando al menos un proceso en un determinado paso de tiempo

Podemos pensar en un proceso como un programa o aplicación en ejecución. Y en este caso una aplicación es un archivo escrito en algún lenguaje de programación que esta siendo ejecutado por el microprocesador.

También puede suceder que en un proceso se puedan ejecutar varias instancias, como cuando se abren varias ventanas de un clock de notas o un navegador. Así, se deduce y es verdad que un proceso contiene un hilo –mínimo el hilo principal que corre en primer plano.

## CPU v.s. GPU

Actualmente los dispositivos de cómputo contienen al menos un *CPU* y dentro de este *CPU* pueden estar contenidos varios núcleos, lo que permite el desarrollo de algoritmos en paralelo.

De igual manera, la mayoría de los dispositivos de cómputo contienen al menos un *GPU* y dentro de este *GPU* pueden existir varios núcleos, la principal diferencia entre ambos (*CPU* y *GPU*) es el propósito para el cuál fueron diseñados

Para fines prácticos (y del curso) podemos pensar que la diferencia principal entre una *CPU* (unidad de procesamiento central) y una *GPU* (**unidad de procesamiento gráfico**) radica en que un *CPU* es un dispositivo de cómputo de propósito general, puede realizar cualquier tipo de cómputo que se le asigne.

Por otro lado un *GPU* esta diseñado para el procesamiento gráfico, lo que significa que la forma en la que procesa información **esta optimizada para trabajar con matrices y vectores**.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/desarrollo/Figuras/MonteCarlo/mejor.png?raw=1" width="600">
</center>

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/desarrollo/Figuras/MonteCarlo/cpu-vs-gpu.jpg?raw=1" width="600">
</center>

# Concurrente v.s. Paralelo

¿Cuál es la diferencia entre concurrencia y
paralelismo?.

Lo primero que hay que decir es: que el paralelismo es una forma de concurrencia; es decir, todo programa paralelo es concurrente.

Una implementación secuencial es la forma tı́pica de solucionar un problema; sin embargo, esta técnica no siempre es la óptima.

El algoritmo secuencial podrá resolver el problema, pero esto no es lo único que nos interesa, sino también en cuánto tiempo lo hace y cuánta memoria necesita. Una desventaja principal es que, no todo algoritmo secuencial puede convertirse a uno paralelo.

En cómputo paralelo en lugar de usar un procesador para solucionar un problema, tendremos $n$ procesadores disponibles resolviendo el mismo problema.

Ası́ el problema se estará resolviendo continuamente, no por intervalos de tiempo, como sucede en la concurrencia, por ello el tiempo invertido en resolver el problema se reduce.

<center>
<img src="https://github.com/jugernaut/ManejoDatos/blob/desarrollo/Imagenes/Simulacion/concurrenteparalelo.png?raw=1" width="800">
</center>

En esta imagen vemos como se distribuye el uso del procesador dependiendo del tipo de algoritmo que se este usando.

<center>
<img src="https://github.com/jugernaut/ManejoDatos/blob/desarrollo/Imagenes/Simulacion/tiempo.png?raw=1" width="800">
</center>

Y finalmente podemos ver la diferencia en tiempo de procesamiento de un algoritmo concurrente y uno en paralelo.

### Conceptos

Ademas de las técnicas previamente mencionadas para poder realizar múltiples tareas dentro de una computadora, existen 2 conceptos relativamente nuevos:

* **Programación en paralelo Multicore**: con la creciente capacidad de cómputo actual existen computadoras que cuentan con más de un núcleo (core) es decir que cuentan con más de un microprocesador. Es en este tipo de equipos en los cuales se pueden ejecutar múltiples procesos (no solo hilos) en paralelo con la finalidad de realizar una tarea en el menor tiempo posible.

* **Programación en paralelo mediante GPU's**: un GPU es una unidad grafica de procesamiento, casi por accidente se descubrió que estas unidades graficas también son útiles para llevar a cabo cómputo en paralelo, incluso han mostrado un mejor desempeño en cálculos de punto flotante que algunos microprocesadores. Sin embargo su desventaja principal es que la memoria no es compartida.

Se sigue de las definiciones anteriores que un proceso se puede ejecutar de manera secuencial o de manera concurrente.

* **Secuencial**: proceso, tarea o algoritmo que tiene que ejecutarse paso a paso, es decir que hasta que no se termine de realizar uno de sus pasos, no se puede iniciar el siguiente.

* **Concurrente**: todo aquel proceso tarea o algoritmo que puede ser dividido en subtareas mas pequeñas que pueden ser realizadas al mismo tiempo. Una vez realizadas las subtareas, los resultados pueden unirse.

# Ley de Amdahl

Sea $f$ la fracción de operaciones en un calculo computacional que sera llevado a cabo de manera secuencial, donde $0\leq f\leq1$. La máxima velocidad $\Psi$ alcanzada mediante programación en paralelo con una computadora con $p$ procesadores enfocados en el mismo calculo es:

$$\Psi\leq\frac{1}{f+(1-f)/p}$$

¿Qué interpretación le das a esta ley?

# *threading*

La biblioteca es un módulo de *Python* que nos permite hacer uso de hilos de manera muy sencilla, basta con importar esta biblioteca y usarla.

La forma más sencilla de usar un `Thread` es crear una instancia con un función de destino y llamar a `start()` para que comience a funcionar.

In [ ]:
import threading

def trabajador():
  # esto es lo único que hace nuestro trabajador
  print('Trabajador')

# se genera una lista de hilos
hilos = []
for i in range(5):
  # agregamos cada hilo a la lista
  t = threading.Thread(target=trabajador)
  hilos.append(t)
  # se ejecuta cada hilo, ¿de forma concurrente o en paralelo?.
  t.start()

Trabajador
Trabajador
Trabajador
Trabajador
Trabajador


Esta forma es la forma más sencilla de hacer uso de hilos, sin embargo se esta realizando de manera funcional, es decir que no estamos haciendo uso de la POO.

Otra forma de hacer uso de los hilos, es mediante POO en la cual vamos a crear una clase que herede de la clase `threading` y que sea capas de funcionar mediante hilos.

Volvamos a ver el ejemplo del carrito de compras, a continuación se muestra la versión secuencial de la misma.

In [ ]:
import time

# clase que ayuda a contener el nombre y articulos que compra un cliente
class Cliente(object):

  def __init__(self, nombre):
    # nombre del cliente
    self.nombre = nombre
    # articulos que compro el cliente
    self.carrito = []

# clase que simula el cobro que llevaria a cabo una empresa de compras en linea
class Amazon(object):

  # VARIABLE DE CLASE O ESTATICA
  # diccionario con el catalogo de articulos y su precio
  catalogo = {'PS5':10000, 'Audifonos':2500, 'Halo':1500, 'HDD':900,
              'Laptop':25000, 'GTX1000':6000, 'SmartWatch':7500}

  def __init__(self, nombre):
    self.nombre = nombre

  # metodo que simula el cobro de un cliente
  def cobrar(self, cliente):
    print('Comenzamos a cobrar al cliente :', cliente.nombre)
    # se revisa el carrito del cliente y el catalogo
    total = 0
    # cada articulo se cobra
    for articulo in cliente.carrito:
      print('Se cobra :',articulo, self.catalogo[articulo])
      # se hace una pausa de 2 segundos para simular el cobro
      time.sleep(2)
      # total almacena el total de los articulos del cliente
      total += self.catalogo[articulo]
    print('Total del cliente', cliente.nombre,':', total)


if __name__ == "__main__":
  inicio = time.time()
  cliente1 = Cliente('Mike')
  cliente1.carrito = ['PS5', 'SmartWatch', 'Audifonos']

  amazon = Amazon('cobrador 1')
  amazon.cobrar(cliente1)

  cliente2 = Cliente('Ivan')
  cliente2.carrito = ['GTX1000', 'HDD', 'GTX1000', 'Laptop']

  amazon.cobrar(cliente2)

  cliente3 = Cliente('Jonathan')
  cliente3.carrito = ['Laptop', 'PS5', 'Audifonos', 'Halo', 'HDD',
                      'GTX1000', 'SmartWatch']

  amazon.cobrar(cliente3)
  fin = time.time()

  print("Este algoritmo tomo: ", (fin-inicio)*1000)


Comenzamos a cobrar al cliente : Mike
Se cobra : PS5 10000
Se cobra : SmartWatch 7500
Se cobra : Audifonos 2500
Total del cliente Mike : 20000
Comenzamos a cobrar al cliente : Ivan
Se cobra : GTX1000 6000
Se cobra : HDD 900
Se cobra : GTX1000 6000
Se cobra : Laptop 25000
Total del cliente Ivan : 37900
Comenzamos a cobrar al cliente : Jonathan
Se cobra : Laptop 25000
Se cobra : PS5 10000
Se cobra : Audifonos 2500
Se cobra : Halo 1500
Se cobra : HDD 900
Se cobra : GTX1000 6000
Se cobra : SmartWatch 7500
Total del cliente Jonathan : 53400
Este algoritmo tomo:  28038.564205169678


Y en la siguiente celda se muestra la versión concurrente (en paralelo si el h.w. lo permite).

In [ ]:
import threading
import time

# clase que ayuda a contener el nombre y articulos que compra un cliente
class Cliente(object):

  def __init__(self, nombre):
    # nombre del cliente
    self.nombre = nombre
    # articulos que compro el cliente
    self.carrito = []

class Amazon(threading.Thread):

  # VARIABLE DE CLASE O ESTATICA
  # diccionario con el catalogo de articulos y su precio
  catalogo = {'PS5':10000, 'Audifonos':2500, 'Halo':1500, 'HDD':900,
              'Laptop':25000, 'GTX1000':6000, 'SmartWatch':7500}

  def __init__(self, nombre, cliente):
    super(Amazon, self).__init__()
    self.nombre = nombre
    self.cliente = cliente

  # metodo que simula el cobro de un cliente
  def run(self):
    print('Comenzamos a cobrar al cliente :', self.cliente.nombre)
    # se revisa el carrito del cliente y el catalogo
    total = 0
    # cada articulo se cobra
    for articulo in self.cliente.carrito:
      time.sleep(2)
      print('Se cobra :',articulo, self.catalogo[articulo])
      # total almacena el total de los articulos del cliente
      total += self.catalogo[articulo]
    print('Total del cliente', self.cliente.nombre,':', total)

if __name__ == "__main__":
  inicio = time.time()
  cliente1 = Cliente('Mike')
  cliente1.carrito = ['PS5', 'SmartWatch', 'Audifonos']

  cliente2 = Cliente('Ivan')
  cliente2.carrito = ['GTX1000', 'HDD', 'GTX1000', 'Laptop']

  cliente3 = Cliente('Jonathan')
  cliente3.carrito = ['Laptop', 'PS5', 'Audifonos', 'Halo', 'HDD',
                      'GTX1000', 'SmartWatch']

  amazon1 = Amazon("Trabajador 1", cliente1)
  # se ejecuta cada hilo de manera concurrente
  amazon1.start()

  amazon2 = Amazon("Trabajador 2", cliente2)
  # se ejecuta cada hilo de manera concurrente
  amazon2.start()

  amazon3 = Amazon("Trabajador 3", cliente3)
  # se ejecuta cada hilo de manera concurrente
  amazon3.start()

  # esperamos a que todos los hilos hayan terminado
  amazon1.join()
  amazon2.join()
  amazon3.join()

  fin = time.time()
  print("Este algoritmo tomo: ", (fin-inicio)*1000)



Comenzamos a cobrar al cliente : MikeComenzamos a cobrar al cliente :
 Ivan
Comenzamos a cobrar al cliente : Jonathan
Se cobra : PS5 Se cobra : GTX1000 6000
10000
Se cobra : Laptop 25000
Se cobra : Se cobra : SmartWatch 7500
HDD 900
Se cobra : PS5 10000
Se cobra :Se cobra : Se cobra : Audifonos 2500
 Audifonos 2500
Total del cliente Mike : 20000
GTX1000 6000
Se cobra : Halo 1500
Se cobra : Laptop 25000
Total del cliente Ivan : 37900
Se cobra : HDD 900
Se cobra : GTX1000 6000
Se cobra : SmartWatch 7500
Total del cliente Jonathan : 53400
Este algoritmo tomo:  14022.159814834595


Vale la pena notar la diferencia en la ejecución de ambos algoritmos y sobretodo la diferencia en el tiempo que tardan ambas versiones.

# Referencias

* Michaell J. Quuin: Parallel Programming in C with OpenMP and MPI.
* GPU Parallel Program Development Using CUDA: Tolga Soyata
* Referencias Libro Web: Introduccion a Python.
* Referencias Dongarra Foster: Source Book of parallel computing.
* [Diferencias CPU Físico v.s. Lógico](https://www.daniloaz.com/es/diferencias-entre-cpu-fisica-cpu-logica-core-nucleo-thread-hilo-y-socket/)